In [ ]:
# default_exp model.rnnvae

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# export

import torch
from torch import nn, optim
from deeptool.architecture import Encoder, Decoder, DownUpConv
from deeptool.utils import Tracker

# RNN VAE

> Structure for an Approach maintained a pseudo space realtion 

In [ ]:
# load some test dataset to confirm architecture:
from deeptool.parameters import get_all_args
from deeptool.dataloader import load_test_batch
args = get_all_args()
args.model_type = "rnnvae"
args.batch_size = 1
args.track = False
batch = load_test_batch(args)
batch["img"].shape

torch.Size([1, 3, 16, 256, 256])

In [ ]:
# export 

def mod_batch(batch, key="img"):
    """
    transform the batch to be compatible with the network by permuting
    """
    if len(batch[key].shape) > 4:
        batch[key] = batch[key][0, :, :, :, :]
        batch[key] = batch[key].permute(1, 0, 2, 3)
    return batch

In [ ]:
batchmod = mod_batch(batch)
batchmod["img"].shape

torch.Size([16, 3, 256, 256])

In [ ]:
# export


class RNN_VAE(nn.Module):

    def __init__(self, device, args):
        """
        The recurrent autoencoder for compressing 3d data.
        It compresses in 2d while (hopefully) maintaining the spatial relation between layers
        """
        super(RNN_VAE, self).__init__()
        self.device = device

        # 1. create the convolutional Encoder
        args.dim = 2
        self.conv_part_enc = DownUpConv(args, pic_size=args.pic_size, n_fea_in=len(
            args.perspectives), n_fea_next=args.n_fea_up, depth=1).to(self.device)

        # save important features
        max_fea, min_size = self.conv_part_enc.max_fea, self.conv_part_enc.min_size
        self.n_z = args.n_z

        self.view_arr = [-1, max_fea * min_size**2]  # as flat vector
        self.view_conv = [-1, max_fea, min_size, min_size]  # as conv block
        self.view_track = [1, len(args.perspectives), -1,
                           args.pic_size, args.pic_size]

        # 2. Apply FC- Encoder Part
        self.fc_part_enc = nn.Sequential(
            nn.Linear(max_fea*min_size*min_size, max_fea*min_size),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(max_fea*min_size, max_fea),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(max_fea, args.n_z),
        ).to(self.device)

        # 3. Transition Layer: GRU
        self.transition = nn.GRU(args.n_z, args.n_z, 1).to(self.device)

        # 4. Apply FC-Decoder Part
        self.fc_part_dec = nn.Sequential(
            nn.Linear(args.n_z, max_fea),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(max_fea, max_fea*min_size),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(max_fea*min_size, max_fea*min_size*min_size),
        ).to(self.device)

        # 5. create the convolutional Decoder
        self.conv_part_dec = DownUpConv(
            args, pic_size=args.pic_size, n_fea_in=len(
                args.perspectives), n_fea_next=args.n_fea_down, depth=1, move='up').to(self.device)

        # the standard loss
        self.mse_loss = nn.MSELoss(reduction='sum')

        # the optimizer
        self.optimizer = optim.Adam(self.parameters(), lr=args.lr)

        # reset the dimension
        args.dim = 3

        # Setup the tracker to visualize the progress
        if args.track:
            self.tracker = Tracker(args)

    def watch_progress(self, test_data, iteration):
        """Outsourced to Tracker"""
        self.tracker.track_progress(self, test_data, iteration)

    def rnn_transition(self, x):
        """
        take the matrix of encoded input slices and apply the RNN part
        """
        # reshape
        x = x.reshape([-1, 1, self.n_z])
        # apply GRU layer
        x, _ = self.transition(x)
        # reshape
        x = x.reshape([-1, self.n_z])
        return x

    def forward(self, batch, update=True):
        """
        calculate the forward pass
        """
        self.zero_grad()

        batch = mod_batch(batch)

        # move to gpu
        img = batch['img'].to(self.device)

        # encode:
        x = self.conv_part_enc(img)
        x = x.reshape(self.view_arr)
        x = self.fc_part_enc(x)

        # apply the GRU transition
        x = self.rnn_transition(x)

        # decode
        x = self.fc_part_dec(x)
        x = x.reshape(self.view_conv)
        x = self.conv_part_dec(x)

        loss = self.mse_loss(img, x)

        #x = x.reshape(self.view_track)

        if update:
            loss.backward()
            self.optimizer.step()

            return x

        else:
            tr_data = {}
            tr_data["loss"] = loss.item()

        return x, tr_data

In [ ]:
batch["img"].shape

torch.Size([16, 3, 256, 256])

In [ ]:
device = torch.device("cuda:0" if (
        torch.cuda.is_available() and args.n_gpu > 0) else "cpu")
rnn_vae = RNN_VAE(device, args)
data = load_test_batch(args)
x, tr = rnn_vae(data, update=False)
x.shape

torch.Size([16, 3, 256, 256])

In [ ]:
batch = load_test_batch(args)
batch["img"].shape

torch.Size([1, 3, 16, 256, 256])

In [ ]:
# hide
from deeptool.train_loop import test_one_batch
from deeptool.parameters import get_all_args, compat_args
args = get_all_args()
args.pic_size = 32

In [ ]:
# 3 dim test
args.model_type = "rnnvae"
args.dim = 3
args = compat_args(args)
test_one_batch(args)

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_dataloader.ipynb.
Converted 01_architecture.ipynb.
Converted 02_utils.ipynb.
Converted 03_parameters.ipynb.
Converted 04_train_loop.ipynb.
Converted 10_diagnosis.ipynb.
Converted 20_dcgan.ipynb.
Converted 21_introvae.ipynb.
Converted 22_vqvae.ipynb.
Converted 23_rnn_vae.ipynb.
Converted 99_index.ipynb.
